## 问题定义

本问题为求解2D Helmholtz方程。其形式可以参考案例18中的PDE定义。


### 求解目标

要求求解给定区域内的Helmholtz方程


## 求解

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra.utils import compose
from modulus.sym.utils.io.plotter import ValidatorPlotter, InferencerPlotter

In [3]:
from sympy import Symbol, pi, sin

import modulus.sym
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.utils.io import csv_to_dict
from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.geometry.primitives_2d import Rectangle
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.sym.domain.validator import PointwiseValidator
from modulus.sym.domain.inferencer import PointwiseInferencer
from modulus.sym.key import Key
from modulus.sym.node import Node
from modulus.sym.eq.pdes.wave_equation import HelmholtzEquation

In [4]:
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 20000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: ${training.rec_results_freq}
  rec_inference_freq: ${training.rec_results_freq}
  rec_monitor_freq: ${training.rec_results_freq}
  rec_constraint_freq: 2000
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: true
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: false
run_mode: train
arch:

### 定义必要组件

#### Geo

In [5]:
x, y = Symbol("x"), Symbol("y")
height = 2
width = 2
rec = Rectangle((-width / 2, -height / 2), (width / 2, height / 2))

#### PDE

In [6]:
wave = HelmholtzEquation(u="u", k=1.0, dim=2)  # 亥姆霍兹方程

#### Model

In [7]:
# 定义简单的全连接网络
# 输入为空间坐标和
# 输出为E_z(x, y)
wave_net = instantiate_arch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u")],
    cfg=cfg.arch.fully_connected,
)

#### Node

In [8]:
nodes = wave.make_nodes() + [wave_net.make_node(name="wave_network")]

#### Domain

In [9]:
domain = Domain()

#### ABC边界

In [10]:
wall = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u": 0},
    batch_size=cfg.batch_size.wall,
)
domain.add_constraint(wall, "wall")

#### PDE约束

In [11]:
interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={
        "helmholtz": -(
            -((pi) ** 2) * sin(pi * x) * sin(4 * pi * y)
            - ((4 * pi) ** 2) * sin(pi * x) * sin(4 * pi * y)
            + 1 * sin(pi * x) * sin(4 * pi * y)
        )
    },  # 非齐次形式
    batch_size=cfg.batch_size.interior,
    bounds={x: (-width / 2, width / 2), y: (-height / 2, height / 2)},
    lambda_weighting={
        "helmholtz": Symbol("sdf"),
    },
)
domain.add_constraint(interior, "interior")

#### 验证器以及其他必要组件

In [12]:
file_path = "openfoam/helmholtz.csv"
if os.path.exists(to_absolute_path(file_path)):
    mapping = {"x": "x", "y": "y", "z": "u"}
    openfoam_var = csv_to_dict(to_absolute_path(file_path), mapping)
    openfoam_invar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["x", "y"]
    }
    openfoam_outvar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["u"]
    }

    openfoam_validator = PointwiseValidator(
        nodes=nodes,
        invar=openfoam_invar_numpy,
        true_outvar=openfoam_outvar_numpy,
        batch_size=1024,
        plotter=ValidatorPlotter(),
    )
    domain.add_validator(openfoam_validator)
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

### 求解器以及求解

In [13]:
# 定义求解器
slv = Solver(cfg, domain)

手动加载日志系统

In [14]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [15]:
slv.solve()

2024-03-01 13:51:31,109 - WARNING - Installed PyTorch version 2.2.0a0+81ea7a4 is not TorchScript supported in Modulus. Version 2.1.0a0+4136153 is officially supported.
2024-03-01 13:51:31,118 - INFO - attempting to restore from: /workspace/20_2D_Helmholtz/outputs
2024-03-01 13:51:31,119 - WARNING - optimizer checkpoint not found
2024-03-01 13:51:31,120 - WARNING - model wave_network.0.pth not found
2024-03-01 13:51:32,621 - INFO - [step:          0] record constraint batch time:  1.195e-01s
2024-03-01 13:51:46,474 - INFO - [step:          0] record validators time:  1.385e+01s
2024-03-01 13:51:46,500 - INFO - [step:          0] saved checkpoint to /workspace/20_2D_Helmholtz/outputs
2024-03-01 13:51:46,502 - INFO - [step:          0] loss:  9.869e+03
2024-03-01 13:51:47,998 - INFO - Attempting cuda graph building, this may take a bit...
2024-03-01 13:52:03,827 - INFO - [step:        100] loss:  9.890e+03, time/iteration:  1.732e+02 ms
2024-03-01 13:52:10,532 - INFO - [step:        200] 

2024-03-01 13:59:42,391 - INFO - [step:       6000] loss:  7.998e-01, time/iteration:  2.199e+02 ms
2024-03-01 13:59:48,968 - INFO - [step:       6100] loss:  1.082e-01, time/iteration:  6.575e+01 ms
2024-03-01 13:59:55,856 - INFO - [step:       6200] loss:  7.399e-01, time/iteration:  6.885e+01 ms
2024-03-01 14:00:02,743 - INFO - [step:       6300] loss:  1.120e-01, time/iteration:  6.885e+01 ms
2024-03-01 14:00:09,250 - INFO - [step:       6400] loss:  2.987e-01, time/iteration:  6.505e+01 ms
2024-03-01 14:00:15,862 - INFO - [step:       6500] loss:  3.611e-01, time/iteration:  6.610e+01 ms
2024-03-01 14:00:22,166 - INFO - [step:       6600] loss:  2.335e-01, time/iteration:  6.301e+01 ms
2024-03-01 14:00:28,839 - INFO - [step:       6700] loss:  1.161e-01, time/iteration:  6.671e+01 ms
2024-03-01 14:00:35,196 - INFO - [step:       6800] loss:  1.533e+00, time/iteration:  6.353e+01 ms
2024-03-01 14:00:41,814 - INFO - [step:       6900] loss:  1.705e-01, time/iteration:  6.614e+01 ms


2024-03-01 14:08:31,061 - INFO - [step:      12800] loss:  3.512e-02, time/iteration:  6.618e+01 ms
2024-03-01 14:08:37,682 - INFO - [step:      12900] loss:  3.693e-02, time/iteration:  6.619e+01 ms
2024-03-01 14:08:58,081 - INFO - [step:      13000] record validators time:  1.335e+01s
2024-03-01 14:08:58,145 - INFO - [step:      13000] saved checkpoint to /workspace/20_2D_Helmholtz/outputs
2024-03-01 14:08:58,157 - INFO - [step:      13000] loss:  4.226e-02, time/iteration:  2.047e+02 ms
2024-03-01 14:09:04,386 - INFO - [step:      13100] loss:  6.506e-02, time/iteration:  6.228e+01 ms
2024-03-01 14:09:10,450 - INFO - [step:      13200] loss:  3.520e-02, time/iteration:  6.062e+01 ms
2024-03-01 14:09:16,647 - INFO - [step:      13300] loss:  4.581e-02, time/iteration:  6.192e+01 ms
2024-03-01 14:09:23,240 - INFO - [step:      13400] loss:  3.312e-02, time/iteration:  6.591e+01 ms
2024-03-01 14:09:29,697 - INFO - [step:      13500] loss:  3.238e-02, time/iteration:  6.454e+01 ms
2024-

2024-03-01 14:19:01,335 - INFO - [step:      19400] loss:  2.754e-02, time/iteration:  1.038e+02 ms
2024-03-01 14:19:13,025 - INFO - [step:      19500] loss:  2.581e-02, time/iteration:  1.169e+02 ms
2024-03-01 14:19:24,197 - INFO - [step:      19600] loss:  2.454e-02, time/iteration:  1.117e+02 ms
2024-03-01 14:19:33,930 - INFO - [step:      19700] loss:  2.506e-02, time/iteration:  9.732e+01 ms
2024-03-01 14:19:41,195 - INFO - [step:      19800] loss:  2.494e-02, time/iteration:  7.258e+01 ms
2024-03-01 14:19:47,156 - INFO - [step:      19900] loss:  2.392e-02, time/iteration:  5.959e+01 ms
2024-03-01 14:19:54,871 - INFO - [step:      20000] record constraint batch time:  1.694e-01s
2024-03-01 14:20:08,721 - INFO - [step:      20000] record validators time:  1.385e+01s
2024-03-01 14:20:08,791 - INFO - [step:      20000] saved checkpoint to /workspace/20_2D_Helmholtz/outputs
2024-03-01 14:20:08,793 - INFO - [step:      20000] loss:  2.929e-02, time/iteration:  2.164e+02 ms
2024-03-01 

### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

![u](./outputs/validators/validator_u.png)